In [1]:
import pandas as pd
from seeq import spy
from seeq.spy.assets import Asset, ItemGroup


In [2]:
WORKBOOK_NAME = 'F1 Telemetry Testing'
DATASOURCE_NAME = 'F1 Telemetry Playground'
SESSION_CONDITION_NAME = 'F1 Sessions'


In [4]:
def get_f1_items():
    scoped_workbook_id = None
    if WORKBOOK_NAME != None:
        scoped_workbook = spy.search({'Name':WORKBOOK_NAME, 'Type':'Workbook'})
        scoped_workbook_id = scoped_workbook['ID'][0]
    
    telemetry_items = spy.search({'Name':'20??.*.*.*','Datasource Name':DATASOURCE_NAME, 'Scoped To':scoped_workbook_id})
    return telemetry_items

In [15]:
class F1_Car(Asset):
    
    @Asset.Attribute()
    def Sectors(self, metadata):
        return metadata[metadata['Name'].str.endswith('Sectors')]
    
    @Asset.Attribute()
    def X_pos(self, metadata):
        return metadata[metadata['Name'].str.endswith('X')]

    @Asset.Attribute()
    def Y_pos(self, metadata):
        return metadata[metadata['Name'].str.endswith('Y')]

    @Asset.Attribute()
    def Z_pos(self, metadata):
        return metadata[metadata['Name'].str.endswith('Z')]

    @Asset.Attribute()
    def RPM(self, metadata):
        return metadata[metadata['Name'].str.endswith('rpm')]

    @Asset.Attribute()
    def Throttle(self, metadata):
        return metadata[metadata['Name'].str.endswith('throttle')]

    @Asset.Attribute()
    def Brake(self, metadata):
        return metadata[metadata['Name'].str.endswith('brake')]

    @Asset.Attribute()
    def Gear(self, metadata):
        return metadata[metadata['Name'].str.endswith('n_gear')]

    @Asset.Attribute()
    def TimestampPOS(self, metadata):
        return metadata[metadata['Name'].str.endswith('timestamp_pos')]

    @Asset.Attribute()
    def Speed(self, metadata):
        return metadata[metadata['Name'].str.endswith('speed')]

    @Asset.Attribute()
    def LapNumber(self, metadata):
        return metadata[metadata['Name'].str.endswith('lap_number')]

    @Asset.Attribute()
    def DRS(self, metadata):
        return metadata[metadata['Name'].str.endswith('drs')]

    @Asset.Attribute()
    def SessionKeyPOS(self, metadata):
        return metadata[metadata['Name'].str.endswith('SessionKey_pos')]

    @Asset.Attribute()
    def Status(self, metadata):
        return metadata[metadata['Name'].str.endswith('Status')]

#####  Calculations

    @Asset.Attribute()
    def Laps(self, metadata):
        return {
            'Type': 'Condition',
            
            # This formula will create a capsule for every change in the lap number value
            'Formula': "$h.toCondition().renameProperty('value','lap number')",
            
            'Formula Parameters': {
                # We can reference the base class lap number
                '$h': self.LapNumber(),
            }
        }
            
    @Asset.Attribute()
    def Sector_1(self, metadata):
        return {
            'Type': 'Condition',
            
            # This formula will create a new condition that filters out only sector 1
            'Formula': "$condition.keep('Sector', isEqualTo(1))",
            
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }

    @Asset.Attribute()
    def Sector_2(self, metadata):
        return {
            'Type': 'Condition',
            
            'Formula': "$condition.keep('Sector', isEqualTo(2))",
            
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }

    @Asset.Attribute()
    def Sector_3(self, metadata):
        return {
            'Type': 'Condition',
            
            'Formula': "$condition.keep('Sector', isEqualTo(3))",
            
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }
    

In [16]:
telemetry_items = get_f1_items()
telemetry_items[['Season', 'Team', 'Car Number', 'Tag Name']] = telemetry_items['Name'].str.split('.', expand=True)
telemetry_items['Build Asset'] = telemetry_items['Car Number'] 
telemetry_items ['Build Path'] = 'F1 Telemetry >> ' + telemetry_items['Season'] + ' >> ' + telemetry_items['Team']
telemetry_items

,Name,Datasource Name,Scoped To,Time,Count,Pages,Result
0,20??.*.*.*,F1 Telemetry Playground,0F010B42-D287-E830-9873-F81D3A9BC073,00:00:00.60,322,1,Success


,ID,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Season,Team,Car Number,Tag Name,Build Asset,Build Path
0,0F010B92-F7E8-73E0-BF4A-4CABCDE2989B,2025.Red Bull Racing.1.timestamp_pos,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Red Bull Racing,1,timestamp_pos,1,F1 Telemetry >> 2025 >> Red Bull Racing
1,0F010B92-F784-6250-8270-2A726A31E673,2025.Red Bull Racing.1.SessionKey_pos,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Red Bull Racing,1,SessionKey_pos,1,F1 Telemetry >> 2025 >> Red Bull Racing
2,0F010C05-FB24-E810-8738-60E278EE37AD,2025.Alpine.10.speed,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Alpine,10,speed,10,F1 Telemetry >> 2025 >> Alpine
3,0F010C5D-6811-71B0-BEAD-855ACE9542B5,2025.McLaren.81.throttle,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,McLaren,81,throttle,81,F1 Telemetry >> 2025 >> McLaren
4,0F011792-5F0F-F9F0-8791-BB02C59A1732,2025.Red Bull Racing.22.speed,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Red Bull Racing,22,speed,22,F1 Telemetry >> 2025 >> Red Bull Racing
...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0F010C57-7A2E-6010-8EE9-6AA1DEB49AD3,2025.Racing Bulls.6.rpm,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Racing Bulls,6,rpm,6,F1 Telemetry >> 2025 >> Racing Bulls
318,0F010C57-7A2B-F900-9B8D-364917BEAA51,2025.Racing Bulls.6.timestamp_pos,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Racing Bulls,6,timestamp_pos,6,F1 Telemetry >> 2025 >> Racing Bulls
319,0F010C57-7A2E-6010-AB55-0DACE5C22637,2025.Racing Bulls.6.drs,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Racing Bulls,6,drs,6,F1 Telemetry >> 2025 >> Racing Bulls
320,0F010C55-093F-7180-BDB9-4C12A2017696,2025.Williams.55.timestamp_pos,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Williams,55,timestamp_pos,55,F1 Telemetry >> 2025 >> Williams


In [17]:
build_df = spy.assets.build(F1_Car, telemetry_items)
build_df

,Build Path,Build Asset,Build Template,Build Result
0,F1 Telemetry >> 2025 >> Red Bull Racing,1,F1_Car,Success
1,F1 Telemetry >> 2025 >> Alpine,10,F1_Car,Success
2,F1 Telemetry >> 2025 >> McLaren,81,F1_Car,Success
3,F1 Telemetry >> 2025 >> Red Bull Racing,22,F1_Car,Success
4,F1 Telemetry >> 2025 >> Mercedes,63,F1_Car,Success
5,F1 Telemetry >> 2025 >> None,62,F1_Car,Success
6,F1 Telemetry >> 2025 >> Racing Bulls,30,F1_Car,Success
7,F1 Telemetry >> 2025 >> Mercedes,12,F1_Car,Success
8,F1 Telemetry >> 2025 >> Kick Sauber,27,F1_Car,Success
9,F1 Telemetry >> 2025 >> Williams,55,F1_Car,Success


,ID,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Season,Team,Car Number,Tag Name,Referenced Name,Reference,Name,Asset,Asset Object,Path,Template,Build Result,Formula,Formula Parameters
0,0F010B92-F6CD-60A0-AC94-9802D43BC103,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Red Bull Racing,1,brake,2025.Red Bull Racing.1.brake,True,Brake,1,F1 Telemetry >> 2025 >> Red Bull Racing >> 1,F1 Telemetry >> 2025 >> Red Bull Racing,F1 Car,Success,NaN,NaN
1,0F010B93-1390-7520-A01D-B982DB688732,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Red Bull Racing,1,drs,2025.Red Bull Racing.1.drs,True,DRS,1,F1 Telemetry >> 2025 >> Red Bull Racing >> 1,F1 Telemetry >> 2025 >> Red Bull Racing,F1 Car,Success,NaN,NaN
2,0F010B92-F5CC-FB10-88B6-29F023CE1214,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Red Bull Racing,1,n_gear,2025.Red Bull Racing.1.n_gear,True,Gear,1,F1 Telemetry >> 2025 >> Red Bull Racing >> 1,F1 Telemetry >> 2025 >> Red Bull Racing,F1 Car,Success,NaN,NaN
3,0F010B9B-4FC8-EE90-B0EF-8425D90A857A,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Red Bull Racing,1,lap_number,2025.Red Bull Racing.1.lap_number,True,LapNumber,1,F1 Telemetry >> 2025 >> Red Bull Racing >> 1,F1 Telemetry >> 2025 >> Red Bull Racing,F1 Car,Success,NaN,NaN
4,NaN,NaN,Condition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laps,1,F1 Telemetry >> 2025 >> Red Bull Racing >> 1,F1 Telemetry >> 2025 >> Red Bull Racing,F1 Car,Success,"$h.toCondition().renameProperty('value','lap n...",{'$h': {'ID': '0F010B9B-4FC8-EE90-B0EF-8425D90...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,0F010C5E-FEB9-71C0-9C6E-7CD5386CC6CE,NaN,StoredSignal,NaN,F1 Telemetry Playground,False,2025,Haas,87,Z,2025.Haas.87.Z,True,Z pos,87,F1 Telemetry >> 2025 >> Haas >> 87,F1 Telemetry >> 2025 >> Haas,F1 Car,Success,NaN,NaN
433,NaN,NaN,Condition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sector 1,87,F1 Telemetry >> 2025 >> Haas >> 87,F1 Telemetry >> 2025 >> Haas,F1 Car,Success,"$condition.keep('Sector', isEqualTo(1))",{'$condition': {'ID': '0F010C5F-5671-62A0-9C5D...
434,NaN,NaN,Condition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sector 2,87,F1 Telemetry >> 2025 >> Haas >> 87,F1 Telemetry >> 2025 >> Haas,F1 Car,Success,"$condition.keep('Sector', isEqualTo(2))",{'$condition': {'ID': '0F010C5F-5671-62A0-9C5D...
435,NaN,NaN,Condition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sector 3,87,F1 Telemetry >> 2025 >> Haas >> 87,F1 Telemetry >> 2025 >> Haas,F1 Car,Success,"$condition.keep('Sector', isEqualTo(3))",{'$condition': {'ID': '0F010C5F-5671-62A0-9C5D...


In [18]:
spy.push(metadata=build_df, workbook=WORKBOOK_NAME)

,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Season,Team,Car Number,Tag Name,Referenced Name,...,Formula,Formula Parameters,Cache Enabled,Referenced ID,Scoped To,Datasource Class,Datasource ID,Data ID,ID,Push Result
0,NaN,CalculatedSignal,NaN,F1 Telemetry Playground,False,2025,Red Bull Racing,1,brake,2025.Red Bull Racing.1.brake,...,$signal,[signal=0F010B92-F6CD-60A0-AC94-9802D43BC103],False,0F010B92-F6CD-60A0-AC94-9802D43BC103,0F010B42-D287-E830-9873-F81D3A9BC073,Seeq Data Lab,Seeq Data Lab,[0F010B42-D287-E830-9873-F81D3A9BC073] {Signal...,0F0117DE-C3FE-7540-AFE4-5DEF97D45B1A,Success
1,NaN,CalculatedSignal,NaN,F1 Telemetry Playground,False,2025,Red Bull Racing,1,drs,2025.Red Bull Racing.1.drs,...,$signal,[signal=0F010B93-1390-7520-A01D-B982DB688732],False,0F010B93-1390-7520-A01D-B982DB688732,0F010B42-D287-E830-9873-F81D3A9BC073,Seeq Data Lab,Seeq Data Lab,[0F010B42-D287-E830-9873-F81D3A9BC073] {Signal...,0F0117DE-C41E-7110-A077-D2EA6CD8C966,Success
2,NaN,CalculatedSignal,NaN,F1 Telemetry Playground,False,2025,Red Bull Racing,1,n_gear,2025.Red Bull Racing.1.n_gear,...,$signal,[signal=0F010B92-F5CC-FB10-88B6-29F023CE1214],False,0F010B92-F5CC-FB10-88B6-29F023CE1214,0F010B42-D287-E830-9873-F81D3A9BC073,Seeq Data Lab,Seeq Data Lab,[0F010B42-D287-E830-9873-F81D3A9BC073] {Signal...,0F0117DE-C434-60A0-A2AB-DA7E64B258C1,Success
3,NaN,CalculatedSignal,NaN,F1 Telemetry Playground,False,2025,Red Bull Racing,1,lap_number,2025.Red Bull Racing.1.lap_number,...,$signal,[signal=0F010B9B-4FC8-EE90-B0EF-8425D90A857A],False,0F010B9B-4FC8-EE90-B0EF-8425D90A857A,0F010B42-D287-E830-9873-F81D3A9BC073,Seeq Data Lab,Seeq Data Lab,[0F010B42-D287-E830-9873-F81D3A9BC073] {Signal...,0F0117DE-C44E-EE50-8902-D15B636DA60E,Success
4,NaN,CalculatedCondition,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"$h.toCondition().renameProperty('value','lap n...",[h=0F0117DE-C44E-EE50-8902-D15B636DA60E],NaN,NaN,0F010B42-D287-E830-9873-F81D3A9BC073,Seeq Data Lab,Seeq Data Lab,[0F010B42-D287-E830-9873-F81D3A9BC073] {Condit...,0F011874-D664-7310-B14A-D7EE2F83643C,Success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
__side_effect_asset_9__,NaN,Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[0F010B42-D287-E830-9873-F81D3A9BC073] {Asset}...,0F0117DE-F81A-62C0-BD8E-D1B4BDD656DC,Success
__side_effect_asset_10__,NaN,Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[0F010B42-D287-E830-9873-F81D3A9BC073] {Asset}...,0F0117DE-F81C-F9D0-B35C-6A3F3F42BD4B,Success
__side_effect_asset_11__,NaN,Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[0F010B42-D287-E830-9873-F81D3A9BC073] {Asset}...,0F0117DE-F81C-F9D0-9DE0-84DEB12904D1,Success
__side_effect_asset_12__,NaN,Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[0F010B42-D287-E830-9873-F81D3A9BC073] {Asset}...,0F0117DE-F81F-60E0-874F-648DD74A1114,Success
